# U-Net

doing classification on every pixel, so the input and output share the same size

In [21]:
# from 5.1 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input, Dropout, Activation
from tensorflow.keras.layers import Conv2DTranspose, MaxPooling2D, Conv2D
from tensorflow.keras.layers import concatenate

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
# imports our layers we want to use

from tensorflow.python.keras.losses import categorical_crossentropy #loss function
from tensorflow.python.keras.optimizers import Adam, SGD #optimisers
from tensorflow.keras.utils import to_categorical #some function for data preparation

from tensorflow.compat.v1 import disable_eager_execution
from tensorflow.python.keras.optimizers import RMSprop
import numpy as np
import tensorflow as tf

In [36]:
# build model

init = tf.keras.initializers.VarianceScaling(scale=2.0, mode='fan_in', distribution='normal', seed=None)

# 4 blocks in contracting path

# want to use padding "valid" because the convolutions have to be unpadded
f = 64
input_layer = Input((572, 572, 1))

# it is necessary to give a name to our layer outputs because we want to use them later in the upscaling again 
# conv1 = model.add(Conv2D(f, (3,3), padding="valid", strides=1, activation="relu", input_shape=(572,572,1)))
# conv1 = model.add(Conv2D(f, (3,3), padding="valid", strides=1, activation="relu"))(conv1)
# pool1= model.add(MaxPooling2D(pool_size=(2,2)))(conv1)
# pool1 = model.add(Dropout(0.25))(pool1)

#1
conv1 = Conv2D(64, (3,3), activation="relu", kernel_initializer=init)(input_layer)
conv1 = Conv2D(64, (3,3), activation="relu", kernel_initializer=init)(conv1)

print(np.shape(conv1))
conv1 = Cropping2D(cropping=((88, 88), (88, 88)))(conv1)
pool1= MaxPooling2D((2,2))(conv1)
pool1 = Dropout(0.25)(pool1)

print(np.shape(pool1))

#2
conv2 = Conv2D(2*f, (3,3), activation="relu", kernel_initializer=init)(pool1)
conv2 = Conv2D(2*f, (3,3), strides=1, activation="relu", kernel_initializer=init)(conv2)
conv2  = Cropping2D(cropping=((40, 40), (40, 40)))(conv2)
pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
pool2 = Dropout(0.25)(pool2)

#3
conv3 = Conv2D(4*f, (3,3), activation="relu", kernel_initializer=init)(pool2)
conv3 = Conv2D(4*f, (3,3), activation="relu", kernel_initializer=init)(conv3)
conv3  = Cropping2D(cropping=((16, 16), (16, 16)))(conv3)
pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
pool3 = Dropout(0.25)(pool3)

#4
conv4 = Conv2D(8*f, (3,3), activation="relu", kernel_initializer=init)(pool3)
conv4 = Conv2D(8*f, (3,3), activation="relu", kernel_initializer=init)(conv4)
print(np.shape(conv4))
conv4  = Cropping2D(cropping=((4, 4), (4, 4)))(conv4)
pool4 = MaxPooling2D(pool_size=(2,2))(conv4)
pool4 = Dropout(0.25)(pool4)

# middle/ bottom part: 
conv_m = Conv2D(16*f, (3,3), padding="valid", activation="relu", kernel_initializer=init)(pool4)
conv_m = Conv2D(16*f, (3,3), padding="valid", activation="relu", kernel_initializer=init)(conv_m)

# expansive part

# we want to upsize the image back to it's original size
# use transposed convolution for this 
# trasposed convolution =/= deconvolution
# transposed convolution is somewhat similar
# it produces the same spatial resolution a hypothetical deconvolutional layer would
# only thing in common is it guarantees that the output will be a 5x5 image as well
# back to original shape 
# this step will not reverse the process from above. At least not concerning the numeric values.

# concatenate: It takes as input a list of tensors, all of the same shape except for the concatenation axis
# and returns a single tensor that is the concatenation of all inputs.
# Concatenate = verknüpfen 

deconv4 = Conv2DTranspose(f*8, (2,2), strides=(2,2), padding="same", kernel_initializer=init)(conv_m)
uconv4 = concatenate([deconv4,conv4])
uconv4 = Conv2D(f*8, (3,3), activation="relu", padding="same")(uconv4)
uconv4 = Conv2D(f*8, (3,3), activation="relu", padding="same")(uconv4)

deconv3 = Conv2DTranspose(f*4, (2,2), padding="valid", kernel_initializer=init)(uconv4)
uconv3 = concatenate([deconv3,conv3])
uconv3 = Conv2D(f*4, (3,3), activation="relu", padding="same")(uconv3)
uconv3 = Conv2D(f*4, (3,3), activation="relu", padding="same")(uconv3)

deconv2 = Conv2DTranspose(f*2, (2,2), padding="same", kernel_initializer=init)(uconv3)
uconv2 = concatenate([deconv3,conv3])
uconv2 = Conv2D(f*2, (3,3), activation="relu", padding="same")(uconv2)
uconv2 = Conv2D(f*2, (3,3), activation="relu", padding="same")(uconv2)

deconv1 = Conv2DTranspose(f, (2,2), padding="same", kernel_initializer=init)(uconv2)
uconv1 = concatenate([deconv3,conv3])
uconv1 = Conv2D(f, (3,3), activation="relu", padding="same")(deconv1)
uconv1 = Conv2D(f, (3,3), activation="relu", padding="same")(deconv1)

# output layer 
outputs = Conv2D(1, (1,1), activation="sigmoid", padding="same")

# compile 
unet = Model(inputs=input_layer, outputs = outputs)
unet.compile(optimizer ='adam', loss='binary_crossentropy', metrics=["accuracy"])
unet.summary()

(None, 568, 568, 64)
(None, 196, 196, 64)
(None, 6, 6, 512)


ValueError: Negative dimension size caused by subtracting 2 from 0 for 'max_pooling2d_93/MaxPool' (op: 'MaxPool') with input shapes: [?,0,0,512].